In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import sklearn
import itertools
import numpy as np

In [13]:
liar=pd.read_csv("train2.tsv", sep='\t')

liar.columns = ['drop', 'ID', 'label', 'statement', 'subject', 'speaker', 'speaker title', 'state', 'party', 'barely_count', 
'false_count', 'half_count', 'mostly_count', 'fire_count', 'location', 'context']


'''
liar2=pd.read_csv("test2.tsv", sep='\t')
liar2.columns = ['drop', 'ID', 'label', 'statement', 'subject', 'speaker', 'speaker title', 'state', 'party', 'barely_count', 'false_count', 'half_count', 'mostly_count', 'fire_count', 'location', 'context']
x_liar2 = liar2['statement']
'''

def int_decode(label):
	if label == 'true':
		label='true'
	elif label == 'half-true':
		label='true'
	elif label == 'barely-true':
		label='false'
	elif label == 'mostly-true':
		label='true'
	elif label == 'false':
		label='false'
	elif label == 'pants-fire':
		label='false'
	return label


label_new = []
label_old = liar['label']
for i in range(0, len(liar)):
	label_new.append(int_decode(label_old[i]))

'''
label_new2 = []
label_old2 = liar2['label']
for i in range(0, len(liar2)):
	label_new2.append(int_decode(label_old2[i]))
'''


In [3]:
def clean_text(text, remove_stopwords = True):
    output = ""
    text = re.sub(r'[^\w\s]','',text).lower() #lower texts
    text = str(text).replace(r'http[\w:/\.]+','') # removing urls
    text = str(text).replace(r'[^\.\w\s]', '') #remove everything but characters and punctuation
    text = str(text).replace(r'\.\.+','.') #replace multiple periods with a single one
    text = str(text).replace(r'\.',' . ') #replace periods with a single one
    text = str(text).replace(r'\s\s+',' ') #replace multiple white space with a single one
    
    output = output + text
    return str(output.strip()) #[1:-3].replace(" ", " ")


In [4]:
x_train = liar['statement'].apply(clean_text)



x_train =df['statement'].apply(clean_text)
df['label'] = df['label'].apply(int_encode)
df.head()

,label,statement
0,0,when did the decline of coal start it started ...
1,0,hillary clinton agrees with john mccain by vot...
2,1,health care reform legislation is likely to ma...
3,0,the economic turnaround started at the end of ...
4,0,the chicago bears have had more starting quart...


In [5]:
X_train_clean, X_test_clean, y_train_clean, y_test_clean = train_test_split(df['statement'], df['label'], test_size=0.20, random_state=42)


In [6]:
train_df_clean = pd.concat([X_train_clean, y_train_clean], axis=1)
print("Shape of training data set: ", train_df_clean.shape)
print("View of data set: ", train_df_clean.head())

Shape of training data set:  (8191, 2)
View of data set:                                                statement  label
550   last year exxon pocketed nearly 47 million eve...      0
7229  say ohio reps marcia fudge marcy kaptur and de...      1
2515  70 percent of the people who voted on june 5 2...      0
4466                           qataris dont really work      1
211   you  said that barack obama was not ready to b...      0


In [7]:
eval_df_clean = pd.concat([X_test_clean, y_test_clean], axis=1)
print("Shape of Eval data set: ", eval_df_clean.shape)

Shape of Eval data set:  (2048, 2)


In [8]:
train_args = {
    'evaluate_during_training': True,
    'logging_steps': 100,
    'num_train_epochs': 2,
    'evaluate_during_training_steps': 100,
    'save_eval_checkpoints': False,
    'train_batch_size': 32,
    'eval_batch_size': 64,
    'overwrite_output_dir': True,
    'fp16': False,
    'wandb_project': "visualization-demo"
}

In [9]:
model_bert = ClassificationModel('bert', 'bert-base-cased', num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

In [10]:
model_bert.train_model(train_df_clean, eval_df=eval_df_clean)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:247: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Running loss: 0.631572

/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:224: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:664: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


Running loss: 0.638805


Running loss: 0.438546



In [11]:
result, model_outputs, wrong_predictions = model_bert.eval_model(eval_df_clean, acc=sklearn.metrics.accuracy_score)

In [12]:
print(result)

{'mcc': 0.2542868909382401, 'tp': 450, 'tn': 859, 'fp': 301, 'fn': 438, 'acc': 0.63916015625, 'eval_loss': 0.6618120558559895}


In [ ]:

test_tweet1 = 'postal service unveils plan to pay debts with new one trillion dollar stamp'
predictions, _ = model_Roberta.predict([test_tweet1])
response_dict = {0: 'True', 1: 'Half true', 2: 'Barely true', 3: 'Mostly true',
                 4: 'False', 5: 'Pants on fire'}

print(test_tweet1)
print("Prediction is: ", response_dict[predictions[0]])
